In [1]:
import pandas as pd
from pybaseball import *

In [2]:
bwar_bat_data = bwar_bat()
bwar_pitch_data = bwar_pitch()


In [ ]:
war_data = pd.DataFrame(columns = ['Team', 'Year', 'WinPercent', 'HadDH', 'TotalAvgWAR', 'BattingAvgWAR', 'PitchingAvgWAR', 'TotalWAR', 'TotalBattingWAR', 'TotalPitchingWAR'])

for year in range(1920, 2021):
    teams = team_ids(year)
    bat_war = bwar_bat_data.loc[(bwar_bat_data['year_ID'] == year) & ((bwar_bat_data['lg_ID'] == 'NL') | (bwar_bat_data['lg_ID'] == 'AL'))]
    pitch_war = bwar_pitch_data.loc[(bwar_pitch_data['year_ID'] == year) & ((bwar_pitch_data['lg_ID'] == 'NL') | (bwar_pitch_data['lg_ID'] == 'AL'))]
    total_war = bat_war.append(pitch_war, ignore_index=True, sort=False)
    for team in teams.itertuples(index=False):
        teamIDBR = team[teams.columns.get_loc("teamIDBR")]
        record_objs = schedule_and_record(year, teamIDBR).iloc[-1]['W-L'].split('-')
        record_objs = list(map(int, record_objs))
        win_percent = record_objs[0] / (record_objs[0] + record_objs[1])
        team_bat_war = bat_war.loc[bat_war['team_ID'] == teamIDBR]
        team_pitch_war = pitch_war.loc[pitch_war['team_ID'] == teamIDBR]
        team_total_war = total_war.loc[total_war['team_ID'] == teamIDBR]
        had_dh = True if team[teams.columns.get_loc("lgID")] == 'AL' and year >= 1973 else False
        row = [teamIDBR, year, win_percent, had_dh, team_bat_war.WAR.mean(), team_bat_war.WAR.mean(), team_pitch_war.WAR.mean(),  team_total_war.WAR.sum(), team_bat_war.WAR.sum(), team_pitch_war.WAR.sum()]
        #print(row)
        war_data.loc[len(war_data.index)] = row
    
war_data.sample(5)

/home/fkatz/baseballWAR/baseballWAR_env/lib/python3.8/site-packages/pybaseball/team_results.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('',1) #not a useful column
/home/fkatz/baseballWAR/baseballWAR_env/lib/python3.8/site-packages/pybaseball/team_results.py:83: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('Streak2',1)


In [ ]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial
import matplotlib.pyplot as plt
import scipy.stats

xcolset = ['WinPercent', 'WinPercent', 'WinPercent', 'WinPercent', 'WinPercent', 'WinPercent']
ycolset = ['TotalAvgWAR', 'BattingAvgWAR', 'PitchingAvgWAR', 'TotalWAR', 'TotalBattingWAR', 'TotalPitchingWAR']
xlabelset = ['Win Percentage', 'Win Percentage', 'Win Percentage', 'Win Percentage', 'Win Percentage', 'Win Percentage']
ylabelset = ['Team Average bWAR', 'Batting Average bWAR', 'Pitching Average bWAR', 'Total Team bWAR', 'Total Batting bWAR', 'Total Pitching bWAR']

for i, (xcol,ycol,xlabel,ylabel) in enumerate(zip(xcolset,ycolset,xlabelset,ylabelset), 1):

    xcolval = war_data[xcol]
    ycolval = war_data[ycol]
    
    offset, slope = Polynomial.fit(xcolval, ycolval, 1).convert().coef
    correlation =  scipy.stats.pearsonr(xcolval, ycolval)
    
    plt.figure(i, figsize=(11,8.5))
    plt.scatter(xcolval, ycolval, s=20,c='#4292c6')
    plt.plot(xcolval, offset + slope * xcolval, '-', c='#08306b')
    #plt.plot([], [], ' ', label=f'R = {correlation[0]}')
    #plt.plot([], [], ' ', label=f'$\rho=100$ = {correlation[1]}')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f'{xlabel} vs {ylabel}')
    
    axes = plt.gca()
    xmin, xmax = axes.get_xlim()
    ymin, ymax = axes.get_ylim()
    textxpos = .05
    textypos = .9
    
    plt.text(((xmax-xmin)*textxpos) + xmin, ((ymax-ymin)*textypos) + ymin, f'correlation = {correlation[0]} \nrho 'r'$\rho=$'f'{correlation[1]}', fontsize = 14)
    
    plt.show()
